In [12]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.11.0


## Data Preprocssing

In [20]:
import os
import music21 as m21
import json
import tensorflow as tf
import keras
import numpy as np

In [7]:

SONGS_PATH = "Data/erk"
ACCEPTABLE_DURATIONS = [0.25, 0.5, 0.75, 1.0, 1.5, 2, 3, 4]
SAVE = "Dataset"
SINGLE_FILE_PATH = "single_file"
LENGTH = 64
DICT_PATH = "dict.json"

def load_songs(datset_path):
    songsList = []
    for path,subdir,files in os.walk(datset_path):
        for file in files:
            if file[-3:] == "krn":
                songsList.append(m21.converter.parse(os.path.join(path, file)))
    return songsList

def filter_durations(song, ACCEPTABLE_DURATIONS):

    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in ACCEPTABLE_DURATIONS:
            return False
    return True

# Changing the music data to time series representation
def encode_song(song, timeStamp = 0.25):

    encoded_list = []
    for event in song.flat.notesAndRests:

        if isinstance (event, m21.note.Note):
            symbol = event.pitch.midi
        
        if isinstance(event, m21.note.Rest):
            symbol = "r"
    
        steps = int(event.duration.quarterLength / timeStamp)
        for step in range(steps):
            if step == 0:
                encoded_list.append(symbol)
            else:
                encoded_list.append("_")

    encoded_list = " ".join(map(str, encoded_list))
    return encoded_list


def transpose(song):

    # music score has multiple parts
    parts = song.getElementsByClass(m21.stream.Part)
    partZero = parts[0].getElementsByClass(m21.stream.Measure)
    key = partZero[0][4]

    # Estimating key incase there isn't one
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")
    
    # Interval for transposition
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == 'minor':
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))
    
    transpose_song = song.transpose(interval)
    return transpose_song
    
    
def preprocess(dataset_path):
    songs = load_songs(dataset_path)
    print(f"# songs loaded = {len(songs)}")
    
    for i, song in enumerate(songs):

        # Filtering durations
        if not filter_durations(song, ACCEPTABLE_DURATIONS):
            continue
        
        # Transpoing to C maj / A min
        song = transpose(song)

         # Encoding
        encoded_song = encode_song(song)

        SAVE_songs = os.path.join(SAVE, str(i))
        with open(SAVE_songs, "w") as fp:
            fp.write(encoded_song)
    
def load(path):
    with open(path, "r") as fp:
        song = fp.read()
        return song

def create_single_file(dataset_path, file_path, sequence_len):
    delimeter = "/ " * sequence_len
    songs = ""

# Adding delimeters after loading the encoded songs
    for path, subdir, files in os.walk(dataset_path):
        for file in files:
            filePath = os.path.join(path, file)
            song = load(filePath)
            songs = songs + song + " " + delimeter
    # Take everything apart from the last character which is a space
    songs = songs[:-1]

    with open(file_path, "w") as fp:
        fp.write(songs)
    
    return songs

def mapping(songs, dict_path):
    
    dict = {}

    songs = songs.split()
    uniqueValues = list(set(songs))

    for i, el in enumerate(uniqueValues):
        dict[el] = i
    
    with open(dict_path, "w") as fp:
        json.dump(dict, fp, indent=2)

# Converting the song to a list of integers
def convert_songs(songs):

    intList = []    

    # Mapping List
    with open(DICT_PATH, "r") as fp:
        dict = json.load(fp)
    
    # Converting string to list
    songs = songs.split()

    # Mapping every symbol in the songs list to int
    for el in songs:
        intList.append(dict[el])
    
    return intList

# Generating training sequences, which are a subsets of time series music representation
def training_sequence(sequence_length):
    
    songs = load(SINGLE_FILE_PATH)
    mapped_songs = convert_songs(songs)

    inputs = []
    targets = []

    # Generating sequences
    # Predicting the next musical event in the melody
    # 64 time steps for each training sample
    total_sequences = len(mapped_songs) - sequence_length
    for i in range(total_sequences):
        # Ex: [1,2,3,4] [i: [1,2] -> t: [3]] [[2,3] -> [4]] 
        inputs.append(mapped_songs[i:i+sequence_length])
        targets.append(mapped_songs[i+sequence_length])
    
    # One Hot Ecnoding
    # Inputs is a 2D array, (total_sequences, sequence_length, unique_elements)
    # [[0,1,2],[1,1,2]] -> [[[1,0,0], [0,1,0], [0,0,1]], []]
    #                           0         1        2
    unique_elements = len(set(mapped_songs))
    inputs = keras.utils.to_categorical(inputs, num_classes=unique_elements)
    targets = np.array(targets)

    return inputs, targets


# def main():
#     preprocess(SONGS_PATH)
#     songs = create_single_file(SAVE, SINGLE_FILE_PATH,  LENGTH)
#     mapping(songs, DICT_PATH)
#     inputs, targets = training_sequence(LENGTH)

# main()


## Training

In [8]:
OUTPUT_UNITS = 38 # Total number of unique symbols
NUM_UNITS = [256] # Number of units in the internal layer and this case there is only one layer
LOSS = "sparse_categorical_crossentropy" # Loss function
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 64 # Amount of samples that the network is gonna see before running back propogation
SAVE_NETWORK_PATH = "Network_model.h5"

In [18]:
def build_network(output_units, num_units, loss, learning_rate):
    
    # Model architecture
    input_layer = keras.layers.Input(shape=(None, output_units))
    x = keras.layers.LSTM(num_units[0])(input_layer)
    x = keras.layers.Dropout(0.2)(x)

    output_layer = keras.layers.Dense(output_units, activation = "softmax")(x)

    model = keras.Model(input_layer, output_layer)
    
    # Compiling Model
    model.compile(loss=loss, 
                optimizer=keras.optimizers.Adam(lr = learning_rate), 
                metrics=["accuracy"])
    model.summary()

    return model


    

In [19]:
def train(ouput_units = OUTPUT_UNITS, num_units = NUM_UNITS, loss = LOSS, learning_rate = LEARNING_RATE):
     # Generating training sequences
     inputs, targets =  training_sequence(LENGTH)

     # Neural Network
     network = build_network(ouput_units, num_units, loss, learning_rate)

     # Training the network
     network.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)

     network.save(SAVE_NETWORK_PATH)

train()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, 38)]        0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               302080    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 38)                9766      
                                                                 
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________


/Users/sb/tensorflow-test/env/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50


2023-02-06 11:11:56.941395: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-06 11:11:58.096527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 11:11:58.354632: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 11:12:01.008455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5660/5660 [==============================] - 168s 29ms/step - loss: 0.6740 - accuracy: 0.7913
Epoch 2/50
5660/5660 [==============================] - 163s 29ms/step - loss: 0.5661 - accuracy: 0.8170
Epoch 3/50
5660/5660 [==============================] - 165s 29ms/step - loss: 0.5282 - accuracy: 0.8291
Epoch 4/50
5660/5660 [==============================] - 165s 29ms/step - loss: 0.5002 - accuracy: 0.8370
Epoch 5/50
5660/5660 [==============================] - 165s 29ms/step - loss: 0.4796 - accuracy: 0.8431
Epoch 6/50
5660/5660 [==============================] - 164s 29ms/step - loss: 0.4621 - accuracy: 0.8483
Epoch 7/50
5660/5660 [==============================] - 165s 29ms/step - loss: 0.4468 - accuracy: 0.8529
Epoch 8/50
5660/5660 [==============================] - 165s 29ms/step - loss: 0.4308 - accuracy: 0.8579
Epoch 9/50
5660/5660 [==============================] - 165s 29ms/step - loss: 0.4192 - accuracy: 0.8613
Epoch 10/50
5660/5660 [==============================] - 165s 29ms